In [ ]:
import os

import matplotlib.pyplot as plt

from bluemira.base.file import get_bluemira_path
from bluemira.base.parameter import ParameterFrame
from bluemira.equilibria import Equilibrium
from bluemira.geometry.coordinates import Coordinates
from bluemira.radiation_transport.advective_transport import ChargedParticleSolver


First we load an up equilibrium

In [ ]:
read_path = get_bluemira_path("equilibria", subfolder="data")
eq_name = "DN-DEMO_eqref.json"
eq_name = os.sep.join([read_path, eq_name])
eq = Equilibrium.from_eqdsk(eq_name)


Now we load a first wall geometry, so that the solver can determine where the flux
surfaces intersect the first wall.

In [ ]:
read_path = get_bluemira_path("radiation_transport/test_data", subfolder="tests")
fw_name = "DN_fw_shape.json"
fw_name = os.sep.join([read_path, fw_name])
with open(fw_name, "r") as file:
    fw_dict = json.load(file)
fw_shape = Coordinates(fw_dict)


Then we define some input `Parameter`s for the solve.

In [ ]:
params = ParameterFrame(
    # fmt: off
    [
        ["P_sep_particle", "Separatrix power", 150, "MW", None, "Input"],
        ["f_p_sol_near", "near scrape-off layer power rate", 0.65, "dimensionless", None, "Input"],
        ["fw_lambda_q_near_omp", "Lambda q near SOL at the outboard", 0.003, "m", None, "Input"],
        ["fw_lambda_q_far_omp", "Lambda q far SOL at the outboard", 0.1, "m", None, "Input"],
        ["fw_lambda_q_near_imp", "Lambda q near SOL at the inboard", 0.003, "m", None, "Input"],
        ["fw_lambda_q_far_imp", "Lambda q far SOL at the inboard", 0.1, "m", None, "Input"],
        ["f_lfs_lower_target", "Fraction of SOL power deposited on the LFS lower target", 0.9 * 0.5, "dimensionless", None, "Input"],
        ["f_hfs_lower_target", "Fraction of SOL power deposited on the HFS lower target", 0.1 * 0.5, "dimensionless", None, "Input"],
        ["f_lfs_upper_target", "Fraction of SOL power deposited on the LFS upper target (DN only)", 0.9 * 0.5, "dimensionless", None, "Input"],
        ["f_hfs_upper_target", "Fraction of SOL power deposited on the HFS upper target (DN only)", 0.1 * 0.5, "dimensionless", None, "Input"],
    ]
    # fmt: on
)


Finally, we initialise the `ChargedParticleSolver` and run it.

In [ ]:
solver = ChargedParticleSolver(params, eq, dx_mp=0.001)
x, z, hf = solver.analyse(first_wall=fw_shape)

solver.plot()
plt.show()